# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [41]:
stocks = pd.read_csv("sp_500_stocks.csv")
#stocks

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#print(data['symbol'])
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'N A RTASAAESE OBSLNGE/TLCMQ)DL(KG', 'calculationPrice': 'close', 'open': None, 'openTime': None, 'openSource': 'aiifclfo', 'close': None, 'closeTime': None, 'closeSource': 'coflaiif', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 140.78, 'latestSource': 'Close', 'latestTime': 'December 28, 2020', 'latestUpdate': 1634121880305, 'latestVolume': None, 'iexRealtimePrice': 142.89, 'iexRealtimeSize': 104, 'iexLastUpdated': 1647684473902, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 135.46, 'previousVolume': 55622966, 'change': 4.76, 'changePercent': 0.03694, 'volume': None, 'iexMarketPercent': 0.000570856988667873, 'iexVolume': 680, 'avgTotalVolume': 110311172, 'iexBidPrice':

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
print(price)
market_cap = data['marketCap']
print(market_cap)

140.78
2400428049372


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ['ticker','stock price','market capitalization','number of shares to buy']
#final_dataframe = pd.DataFrame([[1,3,4,5]],columns=my_columns)
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,ticker,stock price,market capitalization,number of shares to buy


In [7]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
        index = my_columns
    ),
    ignore_index=True

)

,ticker,stock price,market capitalization,number of shares to buy
0,AAPL,140.78,2400428049372,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [8]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe=final_dataframe.append(
    pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ],
        index = my_columns
    ),
    ignore_index=True

    )
    print(stock)

A
AAL
AAP
AAPL
ABBV


In [9]:
final_dataframe

,ticker,stock price,market capitalization,number of shares to buy
0,A,120,37781071566,N/A
1,AAL,16.17,9963554341,N/A
2,AAP,159.6,11246970868,N/A
3,AAPL,141.01,2439108387639,N/A
4,ABBV,104.81,184804029764,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [13]:
stock_group = list(chunks(stocks['Ticker'],100))
stock_strings = []
for i in range(0,len(stock_group)):
    print(i)
    stock_strings.append(','.join(stock_group[i]))
    print(stock_strings[i])
final_dataframe= pd.DataFrame(columns = my_columns)
for stock_string in stock_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?symbols={stock_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    print(batch_api_url)
    #print(data)
    for symbol in stock_string.split(','):
        #print(symbol)
        final_dataframe=final_dataframe.append(
        pd.Series(
        [
        symbol,
        data[symbol]['quote']['latestPrice'],
        data[symbol]['quote']['marketCap'],
        'N/A'
        ],
        index = my_columns
        ),
        ignore_index=True

        )
        
final_dataframe   
    
    #return stock_group[i]

0
A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
1
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
2
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,I

,ticker,stock price,market capitalization,number of shares to buy
0,A,118.37,36348754890,N/A
1,AAL,16.13,9964766696,N/A
2,AAP,163.30,10778177033,N/A
3,AAPL,143.30,2359908810881,N/A
4,ABBV,103.59,183813049291,N/A
5,ABC,98.80,19808549344,N/A
6,ABMD,318.60,14413181565,N/A
7,ABT,111.57,197214004796,N/A
8,ACN,269.88,177809137501,N/A
9,ADBE,503.48,249881438133,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [27]:
portfolio_size = input('Enter the value of your Portfoilio:')
#val=float(portfolio_size)
try:
    val=float(portfolio_size)
except ValueError:
    print('Please enter an Integer')
    portfolio_size = input('Enter the value of your Portfoilio:')
    val=float(portfolio_size)

Enter the value of your Portfoilio:1000000


In [40]:
num_of_apple_shares_to_buy = math.floor(position_size/143.30)
num_of_apple_shares_to_buy

13

In [42]:
position_size=val/len(final_dataframe.index)
position_size
#num_of_apple_shares_to_buy = math.floor(position_size/500)
#num_of_apple_shares_to_buy
for i in range(0,len(final_dataframe.index)):
    #print(i)
    #print(final_dataframe.loc[i,'number of shares to buy'])
    final_dataframe.loc[i,'number of shares to buy'] = math.floor(position_size/final_dataframe.loc[i,'stock price'])
#final_dataframe

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [70]:
writer = pd.ExcelWriter('recommended_trades.xlsx',engine='xlsxwriter')
final_dataframe.to_excel(writer,'Recommended Trades',index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [71]:
background_color = '#FFD700'
font_color = '#000000'

string_format = writer.book.add_format(
    {
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format':'$0.00',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format':'0',
        'font_color':font_color,
        'bg_color':background_color,
        'border':1
    }
)

header_format = writer.book.add_format(
    {
        
        'font_color':'#0000FF',
        'bg_color':'#FF0000',
        'border':1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [72]:
#writer.sheets['Recommended Trades'].set_column('A:A',18,string_format)
#writer.sheets['Recommended Trades'].set_column('B:B',18,dollar_format)
#writer.sheets['Recommended Trades'].set_column('C:C',18,dollar_format)
#writer.sheets['Recommended Trades'].set_column('D:D',18,integer_format)
#writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [73]:
column_format = {
    
    'A':['ticker',string_format],
    'B':['stock price',dollar_format],
    'C':['market capitalizatio',dollar_format],
    'D':['number of shares to buy',integer_format]
}

for column in column_format.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',18,column_format[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_format[column][0],header_format)
writer.save()

## Saving Our Excel Output

Saving our Excel file is very easy: